In [49]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import sqlalchemy
import sqlite3 as sq
import psycopg2

/Users/hayden/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
# Read in Csvs

df = pd.read_csv('resources/Statcast_2020.csv')
df19 = pd.read_csv('resources/Statcast_2019.csv')

# Merge years and picks columns

df_merge = df.append(df19)
df_clean = df_merge[['pitch_type','release_speed','player_name','zone','stand','p_throws','type','balls','strikes','inning','release_spin_rate','release_extension','pitch_number']]
df_clean.head()

,pitch_type,release_speed,player_name,zone,stand,p_throws,type,balls,strikes,inning,release_spin_rate,release_extension,pitch_number
0,FF,98.8,"Fairbanks, Pete",14.0,L,R,X,1,0,9,2483.0,7.2,2
1,FF,98.7,"Fairbanks, Pete",12.0,L,R,B,0,0,9,2522.0,7.2,1
2,SL,89.6,"Fairbanks, Pete",5.0,R,R,S,3,2,9,2537.0,6.9,6
3,FF,100.4,"Fairbanks, Pete",6.0,R,R,S,3,1,9,2469.0,7.1,5
4,FF,97.6,"Fairbanks, Pete",12.0,R,R,S,3,0,9,2339.0,7.1,4


In [16]:
# Prepare pitcher data for sql

# Generate categorical variable list
pitch_cat = df_clean.dtypes[df_clean.dtypes == 'object'].index.tolist()
pitch_cat.remove('type')
pitch_cat.remove('player_name')

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse = False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_clean[pitch_cat]))

# Add the encoded varibale names to the DataFrame
encode_df.columns = enc.get_feature_names(pitch_cat)


# Merge one-hot encoded features and drop the originals
df_encode = df_clean.merge(
    encode_df,
    left_index = True,
    right_index = True
).drop(pitch_cat, 1)

# Drop superfluous columns and add back players_name
df_encode.drop(columns = ['player_name','stand_L'], axis = 1, inplace = True)
df_encode['player_name']=df_clean['player_name'].values

index = [i for i in range(997200)]

df_encode.reset_index(drop=True,inplace=True)

df_encode.head(10)

,release_speed,zone,type,balls,strikes,inning,release_spin_rate,release_extension,pitch_number,pitch_type_CH,...,pitch_type_FT,pitch_type_KC,pitch_type_KN,pitch_type_SI,pitch_type_SL,pitch_type_nan,stand_R,p_throws_L,p_throws_R,player_name
0,98.8,14.0,X,1,0,9,2483.0,7.2,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Fairbanks, Pete"
1,91.1,14.0,S,2,2,9,2347.0,5.8,5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Fairbanks, Pete"
2,98.7,12.0,B,0,0,9,2522.0,7.2,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Fairbanks, Pete"
3,88.2,5.0,S,2,1,9,2366.0,5.7,4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Fairbanks, Pete"
4,89.6,5.0,S,3,2,9,2537.0,6.9,6,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,"Fairbanks, Pete"
5,95.8,11.0,B,1,1,9,2235.0,5.9,3,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,"Fairbanks, Pete"
6,100.4,6.0,S,3,1,9,2469.0,7.1,5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,"Fairbanks, Pete"
7,90.1,13.0,B,0,1,9,2335.0,5.6,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,"Fairbanks, Pete"
8,97.6,12.0,S,3,0,9,2339.0,7.1,4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,"Fairbanks, Pete"
9,98.2,5.0,S,0,0,9,2216.0,6.4,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,"Fairbanks, Pete"


In [17]:
# Create Top Pitcher Databases

df_cole = df_encode[df_encode['player_name'] == 'Cole, Gerrit']
df_degrom = df_encode[df_encode['player_name'] == 'DeGrom, Jacob']
df_scherzer = df_encode[df_encode['player_name'] == 'Scherzer, Max']
df_bauer = df_encode[df_encode['player_name'] == 'Bauer, Trevor']
df_bieber = df_encode[df_encode['player_name'] == 'Bieber, Shane']
df_kershaw = df_encode[df_encode['player_name'] == 'Kershaw, Clayton']
df_lamet = df_encode[df_encode['player_name'] == 'Lamet, Dinelson']
df_nola = df_encode[df_encode['player_name'] == 'Nola, Aaron']
df_buehler = df_encode[df_encode['player_name'] == 'Buehler, Walker']

df_cole.head(10)

,release_speed,zone,type,balls,strikes,inning,release_spin_rate,release_extension,pitch_number,pitch_type_CH,...,pitch_type_FT,pitch_type_KC,pitch_type_KN,pitch_type_SI,pitch_type_SL,pitch_type_nan,stand_R,p_throws_L,p_throws_R,player_name
914,92.9,14.0,B,0,0,3,2424.0,6.8,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,"Cole, Gerrit"
915,96.9,11.0,B,1,1,3,2275.0,5.8,3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,"Cole, Gerrit"
916,96.0,2.0,S,0,2,3,2568.0,5.6,3,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,"Cole, Gerrit"
917,82.2,3.0,S,1,0,3,2612.0,5.4,2,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,"Cole, Gerrit"
918,86.2,13.0,S,0,1,3,2512.0,5.6,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,"Cole, Gerrit"
919,82.0,11.0,B,0,0,3,2581.0,5.2,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,"Cole, Gerrit"
920,85.9,13.0,S,0,0,3,2417.0,5.4,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,"Cole, Gerrit"
921,89.9,8.0,X,1,1,3,1997.0,5.9,3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,"Cole, Gerrit"
922,96.0,6.0,X,2,0,3,2557.0,5.8,3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,"Cole, Gerrit"
923,90.3,13.0,B,0,1,3,2075.0,6.0,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,"Cole, Gerrit"


In [5]:
# Create L and R Handed Databases
df_hand = df_encode
df_hand['p_throws'] = df_clean['p_throws'].values
df_r = df_encode[df_encode['p_throws'] == 'R']
df_l = df_encode[df_encode['p_throws'] == 'L']
df_r = df_r[df_r['release_speed'].isnull() == False]
df_l = df_l[df_l['release_speed'].isnull() == False]
df_r

,release_speed,release_pos_x,release_pos_z,zone,type,balls,strikes,pfx_x,pfx_z,plate_x,...,pitch_name_Fastball,pitch_name_Forkball,pitch_name_Knuckle Curve,pitch_name_Knuckleball,pitch_name_Sinker,pitch_name_Slider,pitch_name_Split-Finger,pitch_name_nan,player_name,p_throws
0,98.8,-0.41,6.46,14.0,X,1,0,-0.15,1.64,1.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Fairbanks, Pete",R
0,91.1,-2.16,6.08,14.0,S,2,2,-0.05,0.40,0.08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Fairbanks, Pete",R
1,98.7,-0.33,6.62,12.0,B,0,0,0.06,1.72,1.61,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Fairbanks, Pete",R
1,88.2,-2.22,6.07,5.0,S,2,1,0.19,0.46,0.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Fairbanks, Pete",R
2,89.6,-0.10,6.79,5.0,S,3,2,0.33,-0.45,0.24,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,"Fairbanks, Pete",R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731779,88.0,-2.87,4.93,13.0,S,0,2,-1.19,0.26,-0.39,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Castillo, Luis",R
731780,96.4,-2.77,5.24,1.0,S,0,2,-1.10,0.92,-0.77,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Castillo, Luis",R
731781,88.6,-2.88,5.10,14.0,S,0,1,-1.17,0.06,0.12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Castillo, Luis",R
731782,95.8,-2.76,5.19,12.0,S,0,0,-1.01,0.98,0.89,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Castillo, Luis",R


In [7]:
# Save Top Pitcher Databases

df_cole.to_csv('resources/cole.csv')
df_degrom.to_csv('resources/degrom.csv')
df_scherzer.to_csv('resources/scherzer.csv')
df_bauer.to_csv('resources/bauer.csv')
df_bieber.to_csv('resources/bieber.csv')
df_kershaw.to_csv('resources/kershaw.csv')
df_lamet.to_csv('resources/lamet.csv')
df_nola.to_csv('resources/nola.csv')
df_buehler.to_csv('resources/buehler.csv')


In [7]:
# Save L and R Handed Pitcher Databases
df_l.to_csv('resources/l.csv')
df_r.to_csv('resources/r.csv')

In [8]:
#Create database sorted by pitchers team
df_teams = df_merge[['inning_topbot','home_team','away_team','pitch_type','release_speed','release_pos_x','release_pos_z','player_name','zone','game_type','stand','p_throws','type','balls','strikes','pfx_x','pfx_z','plate_x','plate_z','inning','vx0','vy0','ax','ay','az','sz_top','sz_bot','effective_speed','release_spin_rate','release_extension','release_pos_y','at_bat_number','pitch_number','pitch_name','spin_axis','delta_home_win_exp','delta_run_exp']]
df_top = df_teams[df_teams['inning_topbot']== 'Top']
df_bottom = df_teams[df_teams['inning_topbot']== 'Bottom']
df_hometeam = df_top.drop(columns = ['away_team','inning_topbot'])
df_awayteam = df_top.drop(columns = ['home_team','inning_topbot'])
df_hometeam['team'] = df_hometeam['home_team']
df_awayteam['team'] = df_awayteam['away_team']
df_teams = df_hometeam.append(df_awayteam)
df_teams = df_teams[df_teams['pitch_type'].isnull() == False]
df_teams = df_teams.drop(columns = ['home_team','away_team'])
df_teams = df_teams.sort_values('team')
df_teams

,pitch_type,release_speed,release_pos_x,release_pos_z,player_name,zone,game_type,stand,p_throws,type,...,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,pitch_name,spin_axis,delta_home_win_exp,delta_run_exp,team
37703,FF,92.5,-1.44,5.56,"Javier, Cristian",11.0,R,L,R,S,...,2278.0,6.0,54.49,19,4,4-Seam Fastball,202.0,0.000,0.000,ARI
30524,CU,75.5,2.47,5.62,"Erlin, Robbie",12.0,R,R,L,S,...,2631.0,5.9,54.64,39,5,Curveball,332.0,0.000,0.000,ARI
30525,FF,90.1,2.40,5.74,"Erlin, Robbie",12.0,R,R,L,B,...,2500.0,6.0,54.54,39,4,4-Seam Fastball,162.0,0.000,0.060,ARI
30526,CU,75.9,2.52,5.49,"Erlin, Robbie",8.0,R,R,L,S,...,2688.0,6.0,54.53,39,3,Curveball,336.0,0.000,-0.078,ARI
30527,FT,90.2,2.60,5.59,"Erlin, Robbie",12.0,R,R,L,B,...,2241.0,6.2,54.28,39,2,2-Seam Fastball,136.0,0.000,0.030,ARI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103985,CU,77.0,3.34,5.55,"Dayton, Grant",13.0,R,R,L,B,...,2523.0,6.6,53.91,47,10,Curveball,337.0,0.000,0.060,WSH
103986,FF,92.2,3.30,5.57,"Dayton, Grant",12.0,R,R,L,S,...,2279.0,7.3,53.20,47,9,4-Seam Fastball,143.0,0.000,0.000,WSH
103987,FF,91.7,3.30,5.65,"Dayton, Grant",12.0,R,R,L,S,...,2281.0,6.5,54.01,47,8,4-Seam Fastball,148.0,0.000,0.000,WSH
103968,SL,82.0,2.21,6.32,"Smith, Will",13.0,R,L,L,B,...,2340.0,6.4,54.12,53,1,Slider,311.0,0.000,0.041,WSH


In [9]:
# Prepare teams data for sql

# Generate categorical variable list
team_cat = df_teams.dtypes[df_teams.dtypes == 'object'].index.tolist()
team_cat.remove('type')
team_cat.remove('player_name')

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse = False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_teams[team_cat]))

# Add the encoded varibale names to the DataFrame
encode_df.columns = enc.get_feature_names(team_cat)


# Merge one-hot encoded features and drop the originals
df_teams_enc = df_teams.merge(
    encode_df,
    left_index = True,
    right_index = True
).drop(team_cat, 1)

# Drop superfluous columns and add back players_name
df_teams_enc.drop(columns = ['player_name','stand_L'], axis = 1, inplace = True)

df_teams_enc.head(10)

,release_speed,release_pos_x,release_pos_z,zone,type,balls,strikes,pfx_x,pfx_z,plate_x,...,team_PHI,team_PIT,team_SD,team_SEA,team_SF,team_STL,team_TB,team_TEX,team_TOR,team_WSH
0,91.1,-2.16,6.08,14.0,S,2,2,-0.05,0.40,0.08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,91.1,-2.16,6.08,14.0,S,2,2,-0.05,0.40,0.08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,98.8,-0.41,6.46,14.0,X,1,0,-0.15,1.64,1.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,98.8,-0.41,6.46,14.0,X,1,0,-0.15,1.64,1.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,88.2,-2.22,6.07,5.0,S,2,1,0.19,0.46,0.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,88.2,-2.22,6.07,5.0,S,2,1,0.19,0.46,0.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,98.7,-0.33,6.62,12.0,B,0,0,0.06,1.72,1.61,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,98.7,-0.33,6.62,12.0,B,0,0,0.06,1.72,1.61,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,95.8,-1.55,6.41,11.0,B,1,1,-0.90,1.42,-1.57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,95.8,-1.55,6.41,11.0,B,1,1,-0.90,1.42,-1.57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Save teams DF
df_teams.to_csv('resources/team.csv')


In [36]:
#Create strike DataFrame
pitcher_strikes = df_clean[['player_name','type','pitch_type']]
pitcher_strikes = pitcher_strikes[pitcher_strikes['type'].isnull() == False]
pitcher_strikes

,player_name,type,pitch_type
0,"Fairbanks, Pete",X,FF
1,"Fairbanks, Pete",B,FF
2,"Fairbanks, Pete",S,SL
3,"Fairbanks, Pete",S,FF
4,"Fairbanks, Pete",S,FF
...,...,...,...
732468,"Fiers, Mike",S,NaN
732469,"Fiers, Mike",S,NaN
732470,"Fiers, Mike",S,NaN
732471,"Fiers, Mike",X,NaN


In [37]:
#Create Zone DataFrame
pitcher_zones = df_clean[['player_name','zone','pitch_type']]
pitcher_zones = pitcher_zones[pitcher_zones['zone'].isnull() == False]
pitcher_zones

,player_name,zone,pitch_type
0,"Fairbanks, Pete",14.0,FF
1,"Fairbanks, Pete",12.0,FF
2,"Fairbanks, Pete",5.0,SL
3,"Fairbanks, Pete",6.0,FF
4,"Fairbanks, Pete",12.0,FF
...,...,...,...
732468,"Fiers, Mike",NaN,NaN
732469,"Fiers, Mike",NaN,NaN
732470,"Fiers, Mike",NaN,NaN
732471,"Fiers, Mike",NaN,NaN


In [38]:
#Save new DataFrames
pitcher_strikes.to_csv('resources/strikes.csv')
pitcher_zones.to_csv('resources/zones.csv')


In [53]:
#SQL JOIN Connection
conn = psycopg2.connect("dbname=team_country_music user=postgres password=** STAND IN PASSWORD **")
cur = conn.cursor()

cur.executescript('''
CREATE TABLE zone_success AS
SELECT strikes.*,
zones.zone
FROM strikes
INNER JOIN zones
ON strikes.id = zones.id;
''')

conn.commit()
conn.close()


ProgrammingError: invalid dsn: missing "=" after "STAND" in connection info string
